# get started with Rafal API requests

In this notebook we will see how to connect to Rafal, get module's and pivot's definitions, and retrieve some data on **sensi** module via a pivot **delta**

## Connection to Rafal

In [ ]:
import json
import pandas as pd
from uuid import uuid4
import requests

In [ ]:
url = "http://review_ops-mongo-deploy-update.the-ica.ninja:8080"
user = "tbonald"
password = "test21"

http://review_ops-mongo-deploy-update.the-ica.ninja:8080/docs/swagger-ui/index.html?url=/assets/swagger.json#

In [ ]:
# login and token retrieval
resp= requests.post(f"{url.rstrip('/')}/login",
                     json={"identifier": user,"password": password},
                    )
token= resp.json()['token']
token

In [ ]:
# headers fields sent to all subsequent Rafal requests
kwcnx = dict(headers={'Authorization': token, "Content-Type": "application/json"})

In [ ]:
# testing connection & server version
resp= requests.get(url.rstrip('/') + '/ping')
print(f"Rafal version = {resp.json()['version']}")
print(f"Rafal DB version = {resp.json()['dbVersion']}")

## Get some definitions 

In [ ]:
# get modules
resp= requests.get(url= f'{url}/modules', **kwcnx)
resp.json()

In [ ]:
# Get module definition for sensi module
module= 'sensi'
resp= requests.get(url= f'{url}/module/{module}/definition', **kwcnx)
resp.json()

In [ ]:
# Get pivots definitions of sensi module
module= 'sensi'
resp= requests.get(url= f'{url}/module/{module}/pivot', **kwcnx)
resp.json()

In [ ]:
# create a dict of pivots def
pivotdef = {pivotdef['name']: pivotdef for pivotdef in resp.json()}
pivotdef.keys()

## Send a pivot request on sensi module

### Data versioning

In [ ]:
# Get branches for the current user and the pivot table
module= 'sensi'
table= pivotdef['delta']['table']
resp= requests.get(url= f'{url}/tables/{table}/branches', **kwcnx)
df= pd.DataFrame.from_records(resp.json()['branches'])[lambda x: x.user == user]
df

In [ ]:
# select an asOf date and convert for versioning
date= '2019-04-12'
versioning = df[df.date == date][['commit', 'date']].rename(columns= {'date': 'asOf'}).to_dict(orient= 'records')
versioning

### Create request parameters

In [ ]:
module= 'sensi'

In [ ]:
# request parameters
my_uuid1= str(uuid4())
reqParam= {
    "metrics": [
        "Sensitivity"
    ],
    "postaggregaggr": {
        
    },
    "with_facet": True,
    "filter_on_query": None,
    "batch": False,
    "batch_description": None,
    "currency": None,
    "pivot": "delta",
    "by": [
        "BusinessLine",
        "Desk",
        "Book",
        "RiskClass"
    ],
    "where": {
        "Date": {
            "values": [
                "2019-04-15"
            ],
            "notValues": [
                
            ],
            "ranges": [
                
            ],
            "dateRanges": [
                
            ],
            "sql": None,
            "match": None
        }
    },
    "versioning": [
        {
            "commit": "e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855",
            "asOf": "2019-04-15"
        }
    ],
    "query_id": my_uuid1
}

### Send request and get results

In [ ]:
# Get pivots definitions of sensi module with a POST http request
resp= requests.post(url= f'{url}/module/{module}/query', json= reqParam, **kwcnx)
resp.text[:500]

In [ ]:
df= pd.read_json(resp.text, orient='records',lines= True)
df